### Regular lab Question – 7

Lab Assignment: LSTM Lab Exercise: Poem Generation

1. Dataset Preparation:
- Download the dataset from Kaggle.
- Load the dataset and explore the columns to understand the structure.
- Concatenate multiple poems into a single text corpus, separating them by newline characters for clarity.

In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import tensorflow as tf

import kagglehub

path = kagglehub.dataset_download("tgdivy/poetry-foundation-poems")

print("Path to dataset files:", path)

100%|██████████| 8.88M/8.88M [00:00<00:00, 50.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tgdivy/poetry-foundation-poems/versions/1


In [2]:
# Load the dataset
df = pd.read_csv(os.path.join(path, "PoetryFoundationData.csv"), nrows=1000)

In [3]:
# Clean data
df['Poem'] = df['Poem'].str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces/newlines with a single space
df['Title'] = df['Title'].str.replace(r'\s+', ' ', regex=True)
df['input'] = df['Title'] + ' *** ' + df['Poem']


# Explore the columns
print(df.columns)

input_data = df['input'].values.tolist()

# Print a portion of the corpus to verify
print(input_data[:3])

Index(['Unnamed: 0', 'Title', 'Poem', 'Poet', 'Tags', 'input'], dtype='object')
[" Objects Used to Prop Open a Window  ***  Dog bone, stapler, cribbage board, garlic press because this window is loose—lacks suction, lacks grip. Bungee cord, bootstrap, dog leash, leather belt because this window had sash cords. They frayed. They broke. Feather duster, thatch of straw, empty bottle of Elmer's glue because this window is loud—its hinges clack open, clack shut. Stuffed bear, baby blanket, single crib newel because this window is split. It's dividing in two. Velvet moss, sagebrush, willow branch, robin's wing because this window, it's pane-less. It's only a frame of air. ", ' The New Church  ***  The old cupola glinted above the clouds, shone among fir trees, but it took him an hour for the half mile all the way up the hill. As he trailed, the village passed him by, greeted him, asked about his health, but everybody hurried to catch the mass, left him leaning against fences, measuring the r

2. Data Preprocessing:
- Convert the text to lowercase and remove special characters or
punctuation if necessary.
- Tokenize the text (e.g., convert each word to a unique integer).
- Use a sliding window to create sequences of words for the LSTM model.
 - For example, if n=5, create sequences of 5 words with the 6th word as the target.
- Pad the sequences so that they all have the same length.

In [4]:
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


# Tokenize the text (convert each word to a unique integer)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_data)
total_words = len(tokenizer.word_index) + 1

print(total_words)

30216


In [5]:
# Create input sequences using sequences of words
input_sequences = []
for line in input_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, min(len(token_list), 50)):  # Cap sequence length to 50
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Calculate max_sequence_len
max_sequence_len = max(len(seq) for seq in input_sequences)

# Pad sequences and create predictors and labels
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = to_categorical(label, num_classes=total_words)

In [6]:
# Define the batch size
batch_size = 32

# Generator function to yield batches of data
def data_generator(predictors, labels):
    dataset_size = len(predictors)
    indices = np.arange(dataset_size)
    np.random.shuffle(indices)
    for idx in indices:
        yield predictors[idx], labels[idx]

# Create a TensorFlow Dataset from the generator function
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(predictors, label),
    output_signature=(
        tf.TensorSpec(shape=(predictors.shape[1],), dtype=tf.int32),
        tf.TensorSpec(shape=(label.shape[1],), dtype=tf.float32)
    )
)

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=10000).batch(batch_size).repeat()

# Split the dataset into training and validation sets
train_size = 100000
val_size = 20000

train_dataset = dataset.take(train_size // batch_size)
val_dataset = dataset.skip(train_size // batch_size).take(val_size // batch_size)

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import pickle

# Define the ModelCheckpoint callback
checkpoint_path = "model_checkpoint.keras"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      monitor='val_loss',
                                      save_best_only=True,
                                      mode='min',
                                      verbose=1)

# Define EarlyStopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

3. LSTM Model Development:
- Define an LSTM model with the following structure:
 - An embedding layer with an appropriate input dimension (based
on vocabulary size) and output dimension (e.g., 100).
 - One or two LSTM layers with 100 units each.

 - A dropout layer with a rate of 0.2 to prevent overfitting.
 - A dense output layer with softmax activation for word prediction.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

def create_model():
    model = Sequential()
    model.add(Embedding(total_words, 50))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(total_words, activation='softmax'))
    return model

model = create_model()
model.build(input_shape=(None, max_sequence_len))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 50)              │       1,510,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100)                 │          60,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 30216)               │       3,051,816 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,623,016 (17.64 MB)

 Trainable params: 4,623,016 (17.64 MB)

 Non-trainable params: 0 (0.00 B)

4. Training:
- Compile the model with categorical cross-entropy as the loss function
and accuracy as the metric.
- Train the model on the sequences for 10-20 epochs (or until it achieves
satisfactory performance).

In [9]:
# Train the model with batching
history = model.fit(train_dataset,
                    epochs=30,
                    verbose=1,
                    validation_data=val_dataset,
                    callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0627 - loss: 7.7042
Epoch 1: val_loss improved from inf to 6.76196, saving model to model_checkpoint.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 112s 32ms/step - accuracy: 0.0627 - loss: 7.7041 - val_accuracy: 0.0747 - val_loss: 6.7620
Epoch 2/30
3122/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0764 - loss: 6.7641
Epoch 2: val_loss improved from 6.76196 to 6.15718, saving model to model_checkpoint.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 95s 29ms/step - accuracy: 0.0764 - loss: 6.7640 - val_accuracy: 0.0966 - val_loss: 6.1572
Epoch 3/30
3123/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0949 - loss: 6.2290
Epoch 3: val_loss improved from 6.15718 to 5.55426, saving model to model_checkpoint.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 99s 31ms/step - accuracy: 0.0949 - loss: 6.2290 - val_accuracy: 0.1230 - val_loss: 5.5543
Epoch 4/30
3124/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1166 - loss: 5.6786
Epoch 4: va

In [10]:
# Save the model
model.save("trained_model.h5")

# Load the trained model
model = tf.keras.models.load_model('trained_model.h5')

5. Text Generation:
- After training, write a function to generate new poetry lines:
 - Start with a seed text (e.g., a short phrase).
 - Predict the next word, append it to the seed text, and use this
updated text to predict the following word.
 - Repeat this process for a specified number of words or lines.
- Generate multiple lines of poetry using different starting phrases.

In [11]:
def generate_poetry(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_texts = ["The sun", "Love's embrace", "Autumn leaves"]
for seed_text in seed_texts:
    generated_poetry = generate_poetry(seed_text, 20, model, max_sequence_len, tokenizer)
    print(f"Generated poetry with seed '{seed_text}':\n{generated_poetry}\n")

Generated poetry with seed 'The sun':
The sun near the camp was a river and the poem is death of the house it takes on a world and

Generated poetry with seed 'Love's embrace':
Love's embrace angelo when i was born a single donkey i work i was at a hurry and do is you shall

Generated poetry with seed 'Autumn leaves':
Autumn leaves when i stood under a days at the office i know that i find my mother what is not so



6. Evaluation and Experimentation:
- Experiment with different LSTM layer sizes, dropout rates, and sequence
lengths to observe their effects on generated text quality.
- Try adding additional LSTM layers and tuning hyperparameters to improve
the creativity or fluency of generated poetry.

In [16]:
# Define the LSTM model
vocab_size = len(tokenizer.word_index) + 1
model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim=100))
model2.add(LSTM(50, return_sequences=True))
model2.add(LSTM(50))
model2.add(Dropout(0.4))  # Dropout layer to prevent overfitting
model2.add(Dense(vocab_size, activation='softmax'))  # Output layer for word prediction

# Compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.build(input_shape=(None, max_sequence_len))
model2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 50, 100)             │       3,021,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 50, 50)              │          30,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 30216)               │       1,541,016 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,613,016 (17.60 MB)

 Trainable params: 4,613,016 (17.60 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
def generate_poetry(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

seed_texts = ["Whispers of night", "A lone star", "In dreams we wander"]
for seed_text in seed_texts:
    generated_poetry = generate_poetry(seed_text, 20, model, max_sequence_len, tokenizer)
    print(f"Generated poetry with seed '{seed_text}':\n{generated_poetry}\n")

Generated poetry with seed 'Whispers of night':
Whispers of night over us in all at night my father is even a young house gone gone just this own dark soft

Generated poetry with seed 'A lone star':
A lone star three and ” if we woke past among his years of each water is my body is law with a

Generated poetry with seed 'In dreams we wander':
In dreams we wander a little patch of night is an house in lie having a maze in shadow but a little body has



### Interpretation
The code utilizes an LSTM model to generate new poetry lines by learning patterns and dependencies in a dataset of poems. It involves preparing the dataset, creating sequences, training an LSTM, and iteratively predicting the next word based on a seed text to generate new text. The goal is to produce text that has similar characteristics to the training data, like style and structure.

By adjusting the model's parameters and experimenting with different seed texts, users can influence the creativity and fluency of the generated poetry.


The generated poetry lines resemble the style and structure of the original dataset because the LSTM model has learned syntactic and thematic patterns from it. When each seed text is provided, the model uses it as a starting point and builds upon it, generating coherent lines that match the tone, rhythm, and sometimes even the sentiment of the original poems.